In [21]:
import pandas as pd
from commcare_export.checkpoint import CheckpointManagerWithDetails
from commcare_export.commcare_hq_client import CommCareHqClient, AUTH_MODE_APIKEY
from commcare_export.commcare_minilinq import get_paginator, PaginationMode
from datetime import datetime
import re
import  numpy as np
# CommCare credentials
username = 'amutenha@zimttech.org'  # Replace with your actual CommCare username
domain = 'zdip-qa'  # Your CommCare project domain
hq_host = 'https://zdip.zimttech.org'  # Your CommCare HQ custom host
API_KEY = '92fbb3f98433a6929a33015273ebe7b38b6dac39'  # Replace with the API key generated in CommCare


zazicSites= pd.read_excel('zazicFacilities.xlsx')



data = pd.ExcelFile('ZDIPlookup.xlsx')

iqaLookup = pd.read_excel('iqaLookup.xlsx')

# lookupTable = pd.read_excel(data,'previousVMMCSubmissions')

# lookupTable.columns = [col.replace('field: ', '') if col.startswith('field: ') else col for col in lookupTable.columns]

# Initialize API client
api_client = CommCareHqClient(hq_host, domain, username, API_KEY, AUTH_MODE_APIKEY)

# Create a paginator for the 'form' resource (fetching data from a form)
form_paginator = get_paginator(resource='form', pagination_mode=PaginationMode.date_modified, page_size=100)
form_paginator.init()

# Initialize checkpoint manager (None used here for simplicity)
checkpoint_manager = CheckpointManagerWithDetails(None, None, PaginationMode.date_modified)

# Date range for filtering
start_date = datetime(2025,6,30)  # 2 December 2024
end_date = datetime(2025, 12,30)  # 6 December 2024

# Specify the app_id to filter forms
desired_app_id ='0778c330a59625e988e36d5a7d0386b6'  


referrals_data = []
AEs_data = []
general_data = []

# Fetch forms iteratively
# Fetch forms using manual pagination
all_forms = []
offset = 0

params = {
    'limit': 100000,
    'app_id': desired_app_id,
    'received_on_start': start_date.strftime('%Y-%m-%dT%H:%M:%S'),
    'received_on_end': end_date.strftime('%Y-%m-%dT%H:%M:%S')
}

forms = api_client.iterate(
    'form',
    form_paginator,
    checkpoint_manager=checkpoint_manager,
    params=params
)

for form in forms:
    all_forms.append(form)
    if len(all_forms) >= 100000:  # Early stopping if desired
        break

In [22]:
# Print a few fetched forms to verify filtering
for form in all_forms[:5]:  
    pass
    # print(f"Received On: {form.get('received_on')}, App ID: {form.get('app_id')}")

filteredResults =[]
for i in all_forms:
    if i["app_id"] ==desired_app_id:
        filteredResults.append(i)

print(filteredResults)
# Filter and categorize forms by app_id and date
for form in forms:
    if form.get('app_id') == desired_app_id:
        # print(form)
        form_received_date = datetime.strptime(form['received_on'], "%Y-%m-%dT%H:%M:%S.%fZ")
        if start_date <= form_received_date <= end_date:
            general_data.append(form)
# Convert each list to a pandas DataFrame
general_df = pd.json_normalize(general_data)
general_df.to_excel('test.xlsx')

general_df.columns = [re.split(r'[./]', col)[-1] for col in general_df.columns]
desiredColumns = [re.split(r'[./]', col)[-1] for col in iqaLookup.columns]
print("desiredcolumns")
print(desiredColumns)

[{'app_id': '0778c330a59625e988e36d5a7d0386b6', 'archived': False, 'attachments': {'form.xml': {'content_type': 'text/xml', 'length': 12532, 'url': 'https://zdip.zimttech.org/a/zdip-qa/api/form_attachment/v1/d98b0e7f-bb90-4f45-a51c-76cfabad2f6c/form.xml'}}, 'build_id': '67775be853d348e79ec9f44c924e7a1d', 'domain': 'zdip-qa', 'edited_by_user_id': None, 'edited_on': None, 'form': {'#type': 'data', '@name': 'ZAZIC OSDV', '@uiVersion': '1', '@version': '669', '@xmlns': 'http://openrosa.org/formdesigner/72526D77-6087-4854-811D-97DBBA955075', 'activity_and_time_period_under_review': {'section': 'e'}, 'copy-1-of-question1': {'InformationCommunicationComment': 'well done. thorough coverage', 'abstinenceRecommendations': 'yes', 'accurateInformationProvided': 'yes', 'communication_to_clients_-_individual_counseling_sessions': 'OK', 'communicationtoClients': 'yes', 'condomUseExplanation': 'yes', 'explanationofBasicHIVfacts': 'yes', 'explanationofHIVandAIDSriskReduction': 'yes', 'explanationofMCMe

In [23]:
# New columns that should NOT be deleted
columns_to_retain = desiredColumns
# Columns that must be numeric (including the new ones)
numeric_columns = [
    
]

# general_df['AERecordingYear'] = pd.to_numeric(general_df['AERecordingYear'])
# Combine the two lists (retain columns that are in either columns_to_retain or numeric_columns)
# columns_to_keep = list(set(columns_to_retain + numeric_columns))
columns_to_keep = columns_to_retain
# Identify missing columns
missing_columns = [col for col in columns_to_keep if col not in general_df.columns]

In [24]:
# Add missing columns to the DataFrame and assign zero
for col in missing_columns:
    general_df[col] = np.nan
# Drop the columns that are not in the columns_to_keep list
# Sort the columns alphabetically in both DataFrames

# For the 'general_df', sort the columns alphabetically
general_df_sorted = general_df[columns_to_keep]

# For the AE-specific columns, we can do the same
# ae_columns = [
#     'aeComments', 'ae_type_code', 'date_ae_identified', 'AERecordingYear', 'AERecordingSite', 
#     'client_age', 'circumcising_cadre', 'ae_classification', 'AERecordingMonth', 'mcMethod','vmmc_number'
# ]
# general_df_sorted['facility'] = general_df_sorted['facility'].replace('Vere', 'Jompani')

# referrals_columns =['ReferrelRecordingMonth', 
#     'ReferrelRecordingSite', 'ReferrelRecordingYear', 'services_referred','reason_for_referral']

general_df_sorted =  general_df_sorted[:-2]
# other_referrals = []
# ae_data_df_sorted = general_df[ae_columns][sorted(ae_columns)]
# ae_data_df_sorted = ae_data_df_sorted[ae_data_df_sorted['AERecordingMonth'].notna()]

# referralsDF_sorted = general_df[referrals_columns][sorted(referrals_columns)]
# referralsDF_sorted = referralsDF_sorted[referralsDF_sorted['ReferrelRecordingMonth'].notna()]
general_df_sorted.to_excel("data.xlsx")

In [25]:
general_df_sorted

,section,District,facility,startingMonth,startingYear,endingMonth,endingYear,clientSpaceSize,spaceCleaniness,waitingSpace,...,starting_month_to_numeric,ending_month_to_numeric,completed_time,started_time,username,username,received_on,form_link,hq_user,month_year_repeat_group
0,e,HURUNGWE,Mwami Rural Hosptal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,NaN,NaN,lmasimba,lmasimba,2025-07-01T18:32:36.306859Z,NaN,NaN,NaN
1,c,HURUNGWE,Mwami Rural Hosptal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,NaN,NaN,lmasimba,lmasimba,2025-07-01T18:32:35.680196Z,NaN,NaN,NaN
2,e,HURUNGWE,Mwami Rural Hosptal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,NaN,NaN,lmasimba,lmasimba,2025-07-01T18:32:35.197130Z,NaN,NaN,NaN
3,b,HURUNGWE,Mwami Rural Hosptal,October,2024,May,2025,NaN,NaN,NaN,...,10,5,NaN,NaN,vmuseza,vmuseza,2025-07-01T17:26:58.658772Z,NaN,NaN,"[{'calculatedMonth': 'October', 'calculatedYea..."
4,a,HURUNGWE,Mwami Rural Hosptal,NaN,NaN,NaN,NaN,yes,yes,yes,...,0,0,NaN,NaN,vmuseza,vmuseza,2025-07-01T17:26:58.108886Z,NaN,NaN,NaN
5,d,HURUNGWE,Mwami Rural Hosptal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,NaN,NaN,vmuseza,vmuseza,2025-07-01T17:26:57.069032Z,NaN,NaN,NaN
6,a,MARONDERA,Masikana Council Clinic,NaN,NaN,NaN,NaN,yes,yes,yes,...,0,0,NaN,NaN,jgavhera,jgavhera,2025-07-01T11:55:23.800665Z,NaN,NaN,NaN
7,c,SANYATI,Jompani,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,NaN,NaN,fgwenzi,fgwenzi,2025-07-01T11:53:37.654981Z,NaN,NaN,NaN
8,e,SANYATI,Jompani,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,NaN,NaN,gguveya,gguveya,2025-07-01T11:23:04.377933Z,NaN,NaN,NaN
9,a,SANYATI,Jompani,NaN,NaN,NaN,NaN,yes,yes,yes,...,0,0,NaN,NaN,gguveya,gguveya,2025-07-01T11:23:03.961382Z,NaN,NaN,NaN
